# Set Fixed Psats

Read default yaml files, write out fixed Psats according to a prescription.

LATS:  CHLAT and SPLAT use average value.
SPLAT:  MF1_1 and MF2_1 averaged, same for MF1_2 and MF2_2

In [1]:
# See what yaml files are locally available.
import glob
for g in glob.glob('*.yaml'):
    print(g)

CHLAT_20210714.yaml
SAT_HF_20220524.yaml
SAT_LF_20211015.yaml
SAT_MF_20211015.yaml
SPLAT_20210714.yaml


In [3]:
# Standard imports
import numpy as np
import yaml
import toml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 20]

# bolo-calc import
from bolo import Top

In [4]:
data = {}
fixed_psats = {}
yamlfiles = {'CHLAT':'CHLAT_20210714',
             'SPLAT':'SPLAT_20210714',
             'SAT1' :'SAT_LF_20211015',
             'SAT2' :'SAT_MF_20211015',
             'SAT3' :'SAT_HF_20220524'
            }

print('Psats derived from optical power and Psat safety factor')
for tel in yamlfiles.keys(): 
    yamlfile = yamlfiles[tel]+'.yaml'
    fixed_psats[yamlfiles[tel]]={}
    dd = yaml.safe_load(open(yamlfile))
    version_date = str(dd['version']['date'])
    telescope = str(dd['version']['name'])
    data[telescope] = {}
    print('{0:8s}  Psat(pW)'.format(telescope))
    del dd['version']

    # location of atmosphere file
    dd['sim_config']['config_dir'] = '../scripts'

    top = Top(**dd)
    top.run()
    
    for chan in top.instrument.tables.keys():
        if chan.split('_')[0] == 'cam' :
            chname = chan.split('_')[2] + '_' + chan.split('_')[3]
            data[telescope][chname] = {}
            #
            # Grab things we're going to print and give them short names and store them in the dictionary.
            psat = top.instrument.tables[chan]['P_sat'][0]
            data[telescope][chname]['psat'] = psat
            
            print('{0:5s} : {1:8.2f}'.format(chname,psat))
    print(' ')


# how many decimals to round to.
ddd = 3

print('===========================================================')
print('LAT Psats (Average of CHLAT and SPLAT for common bands)')
chan = 'ULF_1'
fixed_psats[yamlfiles['SPLAT']][chan]= np.round(data['SPLAT'][chan]['psat'],ddd)
print('{0:5s} : {1:8.3f}'.format(chan,data['SPLAT'][chan]['psat']))
for chan in data['CHLAT'].keys():
    psat_avg = np.round((data['CHLAT'][chan]['psat'] + data['SPLAT'][chan]['psat'])/2,ddd)
    fixed_psats[yamlfiles['CHLAT']][chan]= psat_avg
    fixed_psats[yamlfiles['SPLAT']][chan]= psat_avg
    print('{0:5s} : {1:8.3f}'.format(chan,psat_avg))
    

print('======================')
print('SAT Psats (Average of _1 and _2 in MF bands)')

fixed_psats[yamlfiles['SAT1']]['LF_1']= np.round(data['SAT_LF']['LF_1']['psat'],ddd)
fixed_psats[yamlfiles['SAT1']]['LF_2']= np.round(data['SAT_LF']['LF_2']['psat'],ddd)

avg90 = np.round((data['SAT_MF']['MF1_1']['psat'] + data['SAT_MF']['MF2_1']['psat'])/2,ddd)
fixed_psats[yamlfiles['SAT2']]['MF1_1']= avg90
fixed_psats[yamlfiles['SAT2']]['MF2_1']= avg90
#
avg150 = np.round((data['SAT_MF']['MF1_2']['psat'] + data['SAT_MF']['MF2_2']['psat'])/2,ddd)
fixed_psats[yamlfiles['SAT2']]['MF1_2']= avg150
fixed_psats[yamlfiles['SAT2']]['MF2_2']= avg150

fixed_psats[yamlfiles['SAT3']]['HF_1']= np.round(data['SAT_HF']['HF_1']['psat'],ddd)
fixed_psats[yamlfiles['SAT3']]['HF_2']= np.round(data['SAT_HF']['HF_2']['psat'],ddd)

for chan in fixed_psats[yamlfiles['SAT1']].keys():
    print('{0:5s} : {1:8.3f}'.format(chan,fixed_psats[yamlfiles['SAT1']][chan]))     
for chan in fixed_psats[yamlfiles['SAT2']].keys():
    print('{0:5s} : {1:8.3f}'.format(chan,fixed_psats[yamlfiles['SAT2']][chan]))     
for chan in fixed_psats[yamlfiles['SAT3']].keys():
    print('{0:5s} : {1:8.3f}'.format(chan,fixed_psats[yamlfiles['SAT3']][chan]))     
    

Psats derived from optical power and Psat safety factor
CHLAT     Psat(pW)
LF_1  :     0.81
LF_2  :     4.12
MF_1  :     4.68
MF_2  :    14.19
HF_1  :    37.55
HF_2  :    51.05
 
SPLAT     Psat(pW)
ULF_1 :     0.40
LF_1  :     0.68
LF_2  :     4.24
MF_1  :     4.52
MF_2  :    10.83
HF_1  :    25.84
HF_2  :    33.80
 
SAT_LF    Psat(pW)
LF_1  :     1.55
LF_2  :     6.15
 
SAT_MF    Psat(pW)
MF1_1 :     8.12
MF1_2 :    12.87
MF2_1 :     8.30
MF2_2 :    13.72
 
SAT_HF    Psat(pW)
HF_1  :    27.05
HF_2  :    33.21
 
LAT Psats (Average of CHLAT and SPLAT for common bands)
ULF_1 :    0.403
LF_1  :    0.747
LF_2  :    4.177
MF_1  :    4.598
MF_2  :   12.513
HF_1  :   31.698
HF_2  :   42.426
SAT Psats (Average of _1 and _2 in MF bands)
LF_1  :    1.551
LF_2  :    6.147
MF1_1 :    8.210
MF2_1 :    8.210
MF1_2 :   13.295
MF2_2 :   13.295
HF_1  :   27.048
HF_2  :   33.211


In [5]:
fixed_psats

{'CHLAT_20210714': {'LF_1': 0.747,
  'LF_2': 4.177,
  'MF_1': 4.598,
  'MF_2': 12.513,
  'HF_1': 31.698,
  'HF_2': 42.426},
 'SPLAT_20210714': {'ULF_1': 0.403,
  'LF_1': 0.747,
  'LF_2': 4.177,
  'MF_1': 4.598,
  'MF_2': 12.513,
  'HF_1': 31.698,
  'HF_2': 42.426},
 'SAT_LF_20211015': {'LF_1': 1.551, 'LF_2': 6.147},
 'SAT_MF_20211015': {'MF1_1': 8.21,
  'MF2_1': 8.21,
  'MF1_2': 13.295,
  'MF2_2': 13.295},
 'SAT_HF_20220524': {'HF_1': 27.048, 'HF_2': 33.211}}

# Save fixed psats to toml file.

In [6]:
# Write to toml file
with open('FixedPsats.toml', "w") as toml_file:
    toml.dump(fixed_psats, toml_file, encoder=toml.TomlNumpyEncoder())